In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_columns", None)


In [5]:
transactions = pd.read_csv("QVI_transaction_data.csv")
customers = pd.read_csv("QVI_purchase_behaviour.csv")


In [6]:
transactions.info()
customers.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264836 entries, 0 to 264835
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DATE            264836 non-null  int64  
 1   STORE_NBR       264836 non-null  int64  
 2   LYLTY_CARD_NBR  264836 non-null  int64  
 3   TXN_ID          264836 non-null  int64  
 4   PROD_NBR        264836 non-null  int64  
 5   PROD_NAME       264836 non-null  object 
 6   PROD_QTY        264836 non-null  int64  
 7   TOT_SALES       264836 non-null  float64
dtypes: float64(1), int64(6), object(1)
memory usage: 16.2+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72637 entries, 0 to 72636
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   LYLTY_CARD_NBR    72637 non-null  int64 
 1   LIFESTAGE         72637 non-null  object
 2   PREMIUM_CUSTOMER  72637 non-null  object
dtypes: int64(1), object(2)
m

In [8]:
transactions.head()



,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,43329,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,43330,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8


In [9]:
customers.head()

,LYLTY_CARD_NBR,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,YOUNG SINGLES/COUPLES,Premium
1,1002,YOUNG SINGLES/COUPLES,Mainstream
2,1003,YOUNG FAMILIES,Budget
3,1004,OLDER SINGLES/COUPLES,Mainstream
4,1005,MIDAGE SINGLES/COUPLES,Mainstream


In [10]:
transactions.isnull().sum()



DATE              0
STORE_NBR         0
LYLTY_CARD_NBR    0
TXN_ID            0
PROD_NBR          0
PROD_NAME         0
PROD_QTY          0
TOT_SALES         0
dtype: int64

In [11]:
customers.isnull().sum()

LYLTY_CARD_NBR      0
LIFESTAGE           0
PREMIUM_CUSTOMER    0
dtype: int64

In [15]:
transactions["DATE"] = pd.to_datetime(transactions["DATE"], unit="D", origin="1899-12-30")


ValueError: '0        2018-10-17
1        2019-05-14
2        2019-05-20
3        2018-08-17
4        2018-08-18
            ...    
264831   2019-03-09
264832   2018-08-13
264833   2018-11-06
264834   2018-12-27
264835   2018-09-22
Name: DATE, Length: 264836, dtype: datetime64[ns]' is not compatible with origin='1899-12-30'; it must be numeric with a unit specified

In [16]:
transactions["DATE"] = pd.to_datetime(transactions["DATE"]).dt.date


In [17]:
transactions.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,2018-10-17,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,2019-05-14,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,2019-05-20,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,2018-08-17,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,2018-08-18,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8


In [18]:
transactions["PROD_NAME"].head(20)


0       Natural Chip        Compny SeaSalt175g
1                     CCs Nacho Cheese    175g
2       Smiths Crinkle Cut  Chips Chicken 170g
3       Smiths Chip Thinly  S/Cream&Onion 175g
4     Kettle Tortilla ChpsHny&Jlpno Chili 150g
5     Old El Paso Salsa   Dip Tomato Mild 300g
6     Smiths Crinkle Chips Salt & Vinegar 330g
7        Grain Waves         Sweet Chilli 210g
8      Doritos Corn Chip Mexican Jalapeno 150g
9        Grain Waves Sour    Cream&Chives 210G
10    Smiths Crinkle Chips Salt & Vinegar 330g
11       Kettle Sensations   Siracha Lime 150g
12                    Twisties Cheese     270g
13            WW Crinkle Cut      Chicken 175g
14              Thins Chips Light&  Tangy 175g
15                           CCs Original 175g
16                           Burger Rings 220g
17      NCC Sour Cream &    Garden Chives 175g
18     Doritos Corn Chip Southern Chicken 150g
19                    Cheezels Cheese Box 125g
Name: PROD_NAME, dtype: object

In [19]:
transactions["PROD_NAME"].describe()


count                                     264836
unique                                       114
top       Kettle Mozzarella   Basil & Pesto 175g
freq                                        3304
Name: PROD_NAME, dtype: object

In [20]:
from collections import Counter

# Get unique product names
unique_products = transactions["PROD_NAME"].unique()

# Split into words
all_words = []
for product in unique_products:
    all_words.extend(product.split())


In [28]:
import re

clean_words = []
for word in all_words:
    # Extract alphabetic sequences only (ignore numbers/punctuation)
    matches = re.findall(r'[a-zA-Z]+', word)
    clean_words.extend([m.lower() for m in matches])


In [29]:
word_freq = Counter(clean_words)

# Convert to DataFrame
word_freq_df = (
    pd.DataFrame(word_freq.items(), columns=["Word", "Frequency"])
      .sort_values(by="Frequency", ascending=False)
)

word_freq_df.head(20)


,Word,Frequency
4,g,105
11,chips,21
8,smiths,15
9,crinkle,13
10,cut,13
17,kettle,13
7,cheese,12
22,salt,12
42,original,10
1,chip,9


In [30]:
transactions = transactions[
    ~transactions["PROD_NAME"].str.contains("salsa", case=False)
]


In [31]:
transactions["PROD_NAME"].str.contains("salsa", case=False).sum()


np.int64(0)

In [37]:
import re
from collections import Counter
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
# Optional custom stoplist for known codes/abbreviations
stoplist = {"rrd", "ww"} 

clean_words = []
for word in all_words:
    matches = re.findall(r'[a-zA-Z]+', word)
    for m in matches:
        m = m.lower()
        if len(m) >= 3 and m not in stoplist:
            lemma = lemmatizer.lemmatize(m)
            # Optional manual merge: 'thinly' → 'thins'
            if lemma == "thinly":
                lemma = "thins"
            clean_words.append(lemma)

word_freq = Counter(clean_words)

for word, freq in word_freq.most_common(20):
    print(word, freq)


chip 30
smith 17
crinkle 13
cut 13
kettle 13
cheese 12
salt 12
thins 10
original 10
chicken 8
cream 8
corn 8
pringles 8
chilli 7
doritos 7
sour 6
sea 6
vinegar 5
crisp 5
natural 4


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\icon\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [38]:
transactions.describe()


,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_QTY,TOT_SALES
count,246742.000000,2.467420e+05,2.467420e+05,246742.000000,246742.000000,246742.000000
mean,135.051098,1.355310e+05,1.351311e+05,56.351789,1.908062,7.321322
std,76.787096,8.071528e+04,7.814772e+04,33.695428,0.659831,3.077828
min,1.000000,1.000000e+03,1.000000e+00,1.000000,1.000000,1.700000
25%,70.000000,7.001500e+04,6.756925e+04,26.000000,2.000000,5.800000
50%,130.000000,1.303670e+05,1.351830e+05,53.000000,2.000000,7.400000
75%,203.000000,2.030840e+05,2.026538e+05,87.000000,2.000000,8.800000
max,272.000000,2.373711e+06,2.415841e+06,114.000000,200.000000,650.000000


In [39]:
transactions[transactions["PROD_QTY"] == 200]


,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
69762,2018-08-19,226,226000,226201,4,Dorito Corn Chp Supreme 380g,200,650.0
69763,2019-05-20,226,226000,226210,4,Dorito Corn Chp Supreme 380g,200,650.0


In [40]:
outlier_customer = transactions.loc[
    transactions["PROD_QTY"] == 200, "LYLTY_CARD_NBR"
].iloc[0]

transactions[transactions["LYLTY_CARD_NBR"] == outlier_customer]


,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
69762,2018-08-19,226,226000,226201,4,Dorito Corn Chp Supreme 380g,200,650.0
69763,2019-05-20,226,226000,226210,4,Dorito Corn Chp Supreme 380g,200,650.0


In [41]:
transactions = transactions[transactions["PROD_QTY"] < 100]


In [42]:
word_freq_df.head(15)


,word,frequency
7,175g,26
11,chips,21
20,150g,18
22,&,16
8,smiths,15
16,kettle,13
10,cut,13
9,crinkle,13
21,salt,12
6,cheese,12
